# Graph embedding using SkipGram

This is an embedding of the whole graph, 80/20 training and validation split and all sources

kg-covid-19:
version 20201012

Name: ensmallen-graph
Version: 0.4.4

Name: embiggen
Version: 0.6.0

In [1]:
from pkg_resources import get_distribution
assert(get_distribution("ensmallen-graph").version == '0.4.4')  # identical to 0.4.3 except for addition of some methods like get_edge_id()
assert(get_distribution("embiggen").version == '0.6.0')

In [2]:
import os

exp_name = "80_20_kg_covid_19_20201012_training_test_epoch_500_delta_0.0001_updated_holdouts"
s3_path = "s3://kg-hub-public-data/embeddings/20201012/"  # keep trailing slash

base_dl_dir = "downloaded_data"
graph_data_dir = os.path.join(base_dl_dir, "kg-covid-19-20201012")
embedding_data_dir = os.path.join(base_dl_dir, "embeddings-20201012")

# graph stuff
graph_out_file = os.path.join(graph_data_dir + "/kg-covid-19.tar.gz")
nodes_file = os.path.join(graph_data_dir, "merged-kg_nodes.tsv")
edges_file = os.path.join(graph_data_dir, "merged-kg_edges.tsv")
sorted_edges_file = os.path.join(graph_data_dir, "merged-kg_edges_SORTED.tsv")
graph_tar_url = "https://kg-hub.berkeleybop.io/kg-covid-19/20201012/kg-covid-19.tar.gz"

# embeddings URLs
base_kghub_url = "http://kg-hub.berkeleybop.io/"
embeddings_url = os.path.join(base_kghub_url, "embeddings/20201012/SkipGram_80_20_kg_covid_19_20201012_training_test_epoch_500_delta_0.0001_embedding.npy")
embedding_file = os.path.join(embedding_data_dir, "SkipGram_embedding.npy")

# params
seed = 42
train_percentage = 0.8

In [3]:
import silence_tensorflow.auto # Import needed to avoid TensorFlow warnings and general useless infos.

## Loading the graphs
We load the kg-covid-19 graph from the repository as an undirected graph.

In [4]:
# get the graphs, if necessary

import urllib
import os
os.makedirs(graph_data_dir, exist_ok=True)

if not os.path.exists(nodes_file) or not os.path.exists(edges_file):
    with urllib.request.urlopen(graph_tar_url) as response, \
        open(graph_out_file, 'wb') as out_file:
            data = response.read()  # a `bytes` object
            out_file.write(data)
    os.system("tar -xvzf " + graph_out_file + " -C " + graph_data_dir)

#### only need to do this once, b/c we'll load the sorted.tsv from now on once it is made below

In [5]:
import pandas as pd
edges = pd.read_csv(graph_data_dir + "/merged-kg_edges.tsv", "\t")

/home/jtr4v/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,9,10,11,12,13,14,15,16,17,18,21,22,24,25,26,29,30,31,32,33,34,37,41,43,45,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
sars_cov_2_curie = 'NCBITaxon:2697049'
chembl_prefix = 'CHEMBL.COMPOUND'

In [7]:
chembl_to_sars_cov_2_edges = (
    (edges.subject.str.contains(chembl_prefix) & (edges.object == sars_cov_2_curie)) | 
    (edges.object.str.contains(chembl_prefix) & (edges.subject == sars_cov_2_curie))
)

In [8]:
edges['holdout_edge_label'] = [
    'chembl_to_sars_cov_2' if value else 'normal'
    for value in chembl_to_sars_cov_2_edges]

In [9]:
import os
new_edge_file = os.path.join(graph_data_dir, 'edges_with_holdout_column.tsv')
edges.to_csv(new_edge_file, sep="\t", index=False)

In [11]:
from ensmallen_graph import EnsmallenGraph
graph = EnsmallenGraph.from_unsorted_csv(
    name="kg-covid-19",
    edge_path = new_edge_file,
    sources_column="subject",
    destinations_column="object",
    edge_types_column='holdout_edge_label',
    directed=False,
    node_path = graph_data_dir + "/merged-kg_nodes.tsv",
    nodes_column = 'id',
    node_types_column = 'category',
    default_node_type = 'biolink:NamedThing'
)

In [32]:
graph

The undirected graph Graph has 447766 nodes with 42 different node types:  the 5 most common are biolink:Publication (nodes number 129930), biolink:OntologyClass (nodes number 108266), biolink:Drug (nodes number 32120), biolink:ChemicalSubstance (nodes number 27157) and biolink:Disease (nodes number 24236), of which 8355 are singletons, and 15611957 unweighted edges with 2 different edge types: normal and chembl_to_sars_cov_2, of which 480 are self-loops. The graph is dense as it has a density of 0.01067 and has 9107 connected components, where the component with most nodes has 435728 nodes and the component with the least nodes has 1 nodes. The graph median node degree is 4, the mean node degree is 69.73 and the node degree mode is 1. The top 5 most central nodes are MESH:D014780 (degree 90378), MESH:D006801 (degree 78249), WD:Q30 (degree 65223), MESH:D014777 (degree 54155) and MESH:D017934 (degree 45196).

In [30]:
reduced_graph = graph.remove_components(edge_types=['chembl_to_sars_cov_2'])
reduced_graph = reduced_graph.remove(singletons=True)

In [31]:
reduced_graph

The undirected graph Graph has 435728 nodes with 40 different node types:  the 5 most common are biolink:Publication (nodes number 129492), biolink:OntologyClass (nodes number 104951), biolink:Drug (nodes number 32016), biolink:ChemicalSubstance (nodes number 27152) and biolink:Disease (nodes number 22281) and 15608444 unweighted edges with 2 different edge types: normal and chembl_to_sars_cov_2, of which 314 are self-loops. The graph is dense as it has a density of 0.03548 and is connected, as it has a single component. The graph median node degree is 4, the mean node degree is 71.64 and the node degree mode is 1. The top 5 most central nodes are MESH:D014780 (degree 90378), MESH:D006801 (degree 78249), WD:Q30 (degree 65223), MESH:D014777 (degree 54155) and MESH:D017934 (degree 45196).

## Creating the SkipGram model
We are going to setup the model to use, if available, multiple GPUs.

In [55]:
from cache_decorator import Cache
from tensorflow.distribute import MirroredStrategy
from tensorflow.keras.optimizers import Nadam
from embiggen import SkipGram
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


@Cache(
    cache_path="{cache_dir}/SkipGram/{_hash}_{holdout_idx}.csv.gz",
    cache_dir=embedding_data_dir,
    args_to_ignore=['train_graph']
)
def compute_skipgram_embedding(
    train_graph: EnsmallenGraph,
    holdout_idx: int,
    walk_length: int = 100,
    batch_size: int = 2**9,
    iterations: int = 20,
    return_weight: float = 1.0,
    explore_weight: float = 1.0,
    embedding_size: int = 100,
    window_size: int = 4,
    negative_samples: int = 7,
    patience: int = 6,
    delta: float = 0.1
):
    """Return dataframe with node embedding obtained with SkipGram for train_graph
    
    Given a graph, learn embeddings and return dataframe with node embeddings
    
    Parameters
    -----
    train_graph: EnsmallenGraph
    holdout_idx: int, 
        an int to identify the holdout
    walk_length: int = 100,
        how many nodes for each walk
    batch_size: int = 2**9,
        how many walks for each batch
    iterations: int = 20,
        how many walks per node
    return_weight: float = 1.0,
        node2vec param, equal to 1/p
    explore_weight: float = 1.0,
        node2vec param, equal to 1/q
    embedding_size: int = 100,
        dimensions for embedding
    window_size: int = 4,
        SkipGram window size
    negative_samples: int = 7,
        how many negative samples that NCE function needs to sample
    patience: int = 6,
        how many epochs to wait for loss fxn to improve by [delta]
    delta: float = 0.1
        change in loss fxn to be considered an improvement
        
    Return:
    -------
    pd.DataFrame containing an embedding for each node in train_graph
    """
    training_sequence = Node2VecSequence(
        train_graph,
        walk_length=walk_length,
        batch_size=batch_size,
        iterations=iterations,
        window_size=window_size,
        return_weight=1/p,
        explore_weight=1/q,
        support_mirror_strategy=True
    )

    strategy = MirroredStrategy()
    with strategy.scope():
        model = SkipGram(
            vocabulary_size=train_graph.get_nodes_number(),
            embedding_size=embedding_size,
            window_size=window_size,
            negative_samples=negative_samples,
        )

    history = model.fit(
        training_sequence,
        steps_per_epoch=training_sequence.steps_per_epoch,
        epochs=epochs,
        callbacks=[
            EarlyStopping(
                "loss",
                min_delta=delta,
                patience=patience,
                restore_best_weights=True
            ),
            ReduceLROnPlateau(
                monitor="loss",
                patience=patience//2,
                min_delta=delta
            )
        ]
    )
    return model.get_embedding_dataframe(train_graph.get_nodes_reverse_mapping())

In [ ]:
# https://github.com/monarch-initiative/syntheticLethalityNetwork/tree/embedding_sli/sli_embedding_and_ranking/models

In [ ]:
# make holdouts
from tqdm.auto import trange

for i in trange(5, desc="computing embeddings"):
    pos_training, pos_validation = reduced_graph.connected_holdout(
        train_size=train_percentage, 
        edge_types=['chembl_to_sars_cov_2'],
        random_state=seed+i)
    pos_training.enable_fast_walk()
    embedding = compute_skipgram_embedding(pos_training, i)
    
    
#       transformer = LinkPredictionTransformer(method="hadamard")
#       transformer.fit(embedding)
#       train_X, train_y = transformer.transform(train_pos_sli, train_neg_sli)
#       test_X, test_y = transformer.transform(test_pos_sli, test_neg_sli)
#       for link_prediction_model in link_prediction_models:
#             train_pred, test_pred = link_prediction_model(holdout_number, train_X, train_y, test_X)
#             for run, true, predictions in (
#                 ("train", train_y, train_pred),
#                 ("test", test_y, test_pred)
#             ):
#                 results.append({
#                     "run": run,
#                     "model": link_prediction_model.__name__,
#                     "change_edge_type_weight": change_edge_type_weight,
#                     "holdout_number": holdout_number,
#                     **get_metrics_report(true, predictions)
#                 })

Epoch 1/500
852/852 [==============================] - 422s 495ms/step - loss: 73.8198
Epoch 2/500
639/852 [=====================>........] - ETA: 1:45 - loss: 41.5210

## Upload the weights and embeddings

In [ ]:
from glob import glob
for path in glob(f"{embedding_data_dir}/**/*.csv.gz", recursive=True):
    os.system(f"s3cmd put --acl-public --cf-invalidate {path} {s3_path}")